In [1]:
import numpy as np
import pandas as pd
from sympy import Matrix

In [2]:
def find_tight_rows(_A, _z, _b, epi=1e-8):
    # the following do multiplication of Z with the corresponding A.
    product = np.dot(_A, _z)

    # Check if the product equals b_i for each row. This will also give us a TRUE/FALSE mask of the tight rows
    tight_mask = np.abs(product - _b) < epi #this is done to prevent rounding errors in 3D cases
    # print(tight_mask.shape)
    # print(tight_mask)
    # print(_A.shape)
    A1 = _A[tight_mask.flatten()]
    A2 = _A[~tight_mask.flatten()]
    
#     print("//////")
#     print(_A)
#     print(A1)
#     print(A2)
    return tight_mask, A1, A2

In [3]:
def initial_feasible_vertex(A, b, c, z, dim_n):
    
    bool_mask, A_active, A_inactive = find_tight_rows(A, z, b)
    
    if len(A_active)==0:
        rank = 0
    else:
        rank = np.linalg.matrix_rank(A_active)
    
    if rank != dim_n:
        
        while rank!=dim_n:
            
            cost = np.dot(c, z)
            print(f"z: {tuple(z.flatten())}, Cost: {np.round(cost[0], 2)}")
            
            direction = None
            
            if A_active.shape[0] == 0:
                direction = np.random.rand(untight_rows.shape[-1])
            else:
                null_space_matrix = null_space(tight_rows)
                direction = null_space_matrix[:, 0]
            
            step_magnitudes = []
            for b_, a_ in zip(b[bool_mask], A_inactive):
                step_magnitudes.append((b_ - np.dot(a_, z)) / np.dot(a_, direction))
            
            step_magnitudes = np.array(step_magnitudes)
            step_magnitudes = step_magnitudes[np.where(step_magnitudes > 0)[0]]
            # print(step_magnitudes)
            min_step_magnitude = np.min(step_magnitudes)
            
            z += min_step_magnitude * direction
            
            bool_mask, A_active, A_inactive = find_tight_rows(A, z, b)
            
            if len(A_active)==0:
                rank = 0
            else:
                rank = np.linalg.matrix_rank(A_active)
                
    return z

In [4]:
def find_optimal_vertex(A, b, c, z):
    
    print("\nJourney to an optimal vertex:")
    
    # Identify active constraints where A·z = b
    bool_mask, A_active, A_inactive = find_tight_rows(A, z, b)
    
    # Compute initial reduced costs
    epsilons = -1*np.linalg.inv(A_active).T
    epsilons = epsilons.dot(c)

    while np.any(epsilons > 0):  # While there are negative reduced costs
        beta = 1e-1  # Step size
        cost = np.dot(c.T, z)  # Calculate current objective value
        print(f"z: {tuple(z.flatten())}; Cost: {np.round(cost[0], 2)}")

        # Find the direction of improvement (negative reduced cost)
        A_active_inv = np.linalg.inv(A_active)
        directions = -1*A_active_inv
        
        direction = None
        for i, epsilon in enumerate(epsilons):
            if epsilon > 0:
                direction = directions[:, i:i + 1]
                break
        
        if direction is None:
            print("No direction for improvement found.")
            break        
        
        step_magnitudes = []
        # print(A_inactive)
        # print(z)
        # print(b[~bool_mask])
        for b_, a_ in zip(b[~bool_mask], A_inactive):
            step_magnitudes.append((b_ - np.dot(a_, z)) / np.dot(a_, direction))
        
        # print(direction)
        step_magnitudes = np.array(step_magnitudes)
        # print(step_magnitudes)
        step_magnitudes = step_magnitudes[np.where(step_magnitudes > 0)[0]]
        
        min_step_magnitude = np.min(step_magnitudes)
        # print(min_step_magnitude)
        
        z = z.astype(float)
        z += min_step_magnitude * direction
        
        bool_mask, A_active, A_inactive = find_tight_rows(A, z, b)
        epsilons = -1*np.linalg.inv(A_active).T
        epsilons = epsilons.dot(c)

    # Print the final optimal vertex and cost
    cost = np.dot(c.T, z)
    print(f"Optimal Vertex: z = {tuple(z.flatten())}; Cost = {np.round(cost[0], 2)}")

    return z


In [5]:
def main():
    file_name = 'asg_1_test_case.csv'
    problem_stmt = pd.read_csv(file_name, header=None)
    
    z = problem_stmt.iloc[0, :-1].values.reshape(-1, 1)
    c = problem_stmt.iloc[1, :-1].values
    b = problem_stmt.iloc[2:, -1:].values
    A = problem_stmt.iloc[2:, :-1].values
    
    # print(z.shape)
    # print(c.shape)
    # print(b.shape)
    # print(A.shape)
    # print("///")
    
    initital_neighbour = initial_feasible_vertex(A, b, c, z, len(c))
    # print(initital_neighbour)
    
    optimal_vertex = find_optimal_vertex(A, b, c, initital_neighbour)
    
    # z_optimal, z_cost_all, z_all = vertex_to_vertex_assign1(A, b, z, c, len(c))
    
    #print(optimal_vertex)

In [6]:
main()
 
# Test input for above Output

# 5,0,0,0
# 2,-1,2,0
# 2,1,0,10
# 1,2,-2,20
# 0,1,2,5
# -1,0,0,0
# 0,-1,0,0
# 0,0,-1,0


Journey to an optimal vertex:
z: (5, 0, 0); Cost: 10
Optimal Vertex: z = (5.0, 0.0, 2.5); Cost = 15.0


/var/folders/c2/8jzgq8jj1l3600fxnvmzkg0w0000gn/T/ipykernel_7070/2238840791.py:36: RuntimeWarning: divide by zero encountered in divide
  step_magnitudes.append((b_ - np.dot(a_, z)) / np.dot(a_, direction))
